In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bz2 #can handle .bz2 compressed files

# Exercise 1

## Problem statement
*Count the number of lines in Python for each file.*

## Solution

In [3]:
#Filenames declaration
fsearches='searches.csv.bz2'
fbookings='bookings.csv.bz2'

In [ ]:
# Read the file line by line and count
count=0
with bz2.BZ2File(fsearches,"r") as file:
    for line in file:
        count +=1
print count
nlines_searches = count
# Read the file line by line and count
count=0
with bz2.BZ2File(fbookings,"r") as file:
    for line in file:
        count +=1
print count
nlines_bookings=count

In [7]:
print "File %s has %s lines."%(fsearches,nlines_searches)
print "File %s has %s lines."%(fbookings,nlines_bookings)

File searches.csv.bz2 has 20390199 lines.
File bookings.csv.bz2 has 10000011 lines.


# Exercise 2

## Problem statement

*top 10 arrival airports in the world in 2013 (using the bookings file)*

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)

## Solution

Prepare dataframe to be read using chunks (due to RAM restrictions). Note that the columns are separated by '^' character in the Bookings file.

We will define two different chunksizes:

    * chksize_big: for production
    * chksize_small: for testing and overview purposes.
    
Read column labels and sample (first chunk) to overview data.

In [49]:
chksize_big=1000000 #used for production
chksize_small = 10 #used for testing
reader = pd.read_csv(fbookings,compression='bz2', sep='^',iterator=True, chunksize=chksize_small)
#Get some relevant information about the dataframe
for df in reader:
    [rows,cols] = df.shape
    bookings_labels = df.columns
    bookings_head   = df.head()
    break
#Show sample
print bookings_head

   act_date             source  pos_ctry                          pos_iata  \
0  2013-03-05 00:00:00  1A      DE        a68dd7ae953c8acfb187a1af2dcbe123   
1  2013-03-26 00:00:00  1A      US        e612b9eeeee6f17f42d9b0d3b79e75ca   
2  2013-03-26 00:00:00  1A      US        e612b9eeeee6f17f42d9b0d3b79e75ca   
3  2013-03-26 00:00:00  1A      AU        0f984b3bb6bd06661c95529bbd6193bc   
4  2013-03-26 00:00:00  1A      AU        0f984b3bb6bd06661c95529bbd6193bc   

                          pos_oid                      rloc            \
0  1a11ae49fcbf545fd2afc1a24d88d2b7  ea65900e72d71f4626378e2ebd298267   
1  7437560d8f276d6d05eeb806d9e7edee  737295a86982c941f1c2da9a46a14043   
2  7437560d8f276d6d05eeb806d9e7edee  737295a86982c941f1c2da9a46a14043   
3  36472c6dbaf7afec9136ac40364e2794  5ecf00fdcbcec761c43dc7285253d0c1   
4  36472c6dbaf7afec9136ac40364e2794  5ecf00fdcbcec761c43dc7285253d0c1   

   cre_date             duration  distance  dep_port    ...      \
0  2013-02-22 00:00:00   

In [44]:
print bookings_labels

Index([u'act_date           ', u'source', u'pos_ctry', u'pos_iata',
       u'pos_oid  ', u'rloc          ', u'cre_date           ', u'duration',
       u'distance', u'dep_port', u'dep_city', u'dep_ctry', u'arr_port',
       u'arr_city', u'arr_ctry', u'lst_port', u'lst_city', u'lst_ctry',
       u'brd_port', u'brd_city', u'brd_ctry', u'off_port', u'off_city',
       u'off_ctry', u'mkt_port', u'mkt_city', u'mkt_ctry', u'intl',
       u'route          ', u'carrier', u'bkg_class', u'cab_class',
       u'brd_time           ', u'off_time           ', u'pax', u'year',
       u'month', u'oid      '],
      dtype='object')
